# Eazydiner Booking Predictor

## Loading dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel ('Sample_CUSTOMER_Dataset_2.xlsx')

ddff = pd.read_excel ('Sample_BOOKINGS_Dataset_1.xlsx')

## Deleting unuseful colums  

In [3]:
df.drop(['name','gender'],axis='columns', inplace=True)
ddff.drop(['medium','location','restaurant_name'],axis='columns', inplace=True)

In [4]:
print(df.isnull().sum())
print(ddff.isnull().sum())

cust_id            0
city           17733
AGE                0
medium             0
signup_date        0
dtype: int64
date       1
cust_id    0
dtype: int64


In [5]:
newdf = df.dropna()
newddff = ddff.dropna()

In [6]:
finaldf = newddff.merge(newdf,how="inner",on="cust_id")
finaldf.drop('cust_id',axis='columns', inplace=True)
finaldf

,date,city,AGE,medium,signup_date
0,2020-09-04,Delhi NCR,72,web,2020-09-01
1,2020-09-06,Kolkata,60,google,2020-09-01
2,2020-09-01,Delhi NCR,27,google,2020-09-01
3,2020-09-04,Hyderabad,69,google,2020-09-01
4,2020-09-06,Bengaluru,38,google,2020-09-01
...,...,...,...,...,...
61327,2020-12-01,Hyderabad,38,google,2020-11-30
61328,2020-12-01,Mumbai,26,google,2020-11-30
61329,2020-12-04,Jaipur,31,web,2020-11-30
61330,2020-12-11,Mumbai,18,web,2020-11-30


## Creating label column

In [7]:
finaldf.date = pd.to_datetime(finaldf.date)
finaldf.signup_date= pd.to_datetime(finaldf.signup_date)
diff = (finaldf.date - finaldf.signup_date)
finaldf.drop(['signup_date','date'],axis='columns', inplace=True)
diff = np.where(diff <= '7 days 00:00:00', 1, 0)


In [8]:
labels = diff
labels

array([1, 1, 1, ..., 1, 0, 1])

## One hot encoding our categorical data

In [9]:
dummies = pd.get_dummies(finaldf.medium)
dummies2 = pd.get_dummies(finaldf.city)
finaldf.drop(['medium','city'],axis = 1,inplace = True)

In [10]:
newdata = pd.concat([finaldf,dummies],axis = 'columns')
finaldata = pd.concat([newdata,dummies2],axis = 'columns')

In [11]:
finaldata

,AGE,android,api_partner,concierge,google,ios,web,Abu Dhabi,Agra,Ahmedabad,...,Thrissur,Tirupati,Tirupur,Trichy,Udaipur,Vadodara,Varanasi,Vellore,Vijayawada,Visakhapatnam
0,72,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,69,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,38,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61327,38,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61328,26,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61329,31,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61330,18,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Train test set splitting

In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(finaldata, labels, test_size=0.25, random_state = 100, shuffle = True)

In [31]:

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
threshold = 0.2
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## Model testing

In [32]:
#np.set_printoptions(threshold=np. inf)

In [33]:
y_pred = (logisticRegr.predict_proba(x_test)[:,1] >= threshold).astype(int)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)*100

88.97802126133176